In [1]:
import imagej
from scyjava import jimport
from scipy import stats
import xarray as xr
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from scyjava import jimport, config
import xlsxwriter
from openpyxl import Workbook
from openpyxl import load_workbook
from glob import glob
import os
import re

In [2]:
if 'ij' in locals() and ij is not None:
    ij.dispose()
plugins_dir = r'C:\Program Files\Fiji.app\plugins'
config.add_option(f'-Dplugins.dir={plugins_dir}')
ij = imagej.init(r"C:\Program Files\Fiji.app", mode=imagej.Mode.INTERACTIVE)
print(ij.getVersion())
ij.ui().showUI()

#import imageJ
ImagePlus=jimport('ij.ImagePlus')
IJ = jimport('ij.IJ')
FileSaver = jimport('ij.io.FileSaver')
WaitForUserDialog = jimport('ij.gui.WaitForUserDialog')
ChannelSplitter = jimport('ij.plugin.ChannelSplitter')
ImageCalculator = jimport('ij.plugin.ImageCalculator')
NonBlockingGenericDialog = jimport('ij.gui.NonBlockingGenericDialog')
PointRoi = jimport('ij.gui.PointRoi')
GenericDialog = jimport('ij.gui.GenericDialog')
RoiManager = jimport('ij.plugin.frame.RoiManager')
WindowManager = jimport('ij.WindowManager')
Duplicator = jimport('ij.plugin.Duplicator')
ResultsTable = jimport('ij.measure.ResultsTable')
ImageStack = jimport('ij.ImageStack')

2.14.0/1.54f


In [4]:
path = r"D:\PhD Pauline\µmanager\231206 TDP74\Lamelle 2 - 5Fm-emGFP-TDP43 + FK506"
files_paths = glob(os.path.join(path, "**", "*.tif"), recursive=True)
dre = re.compile(r'(\d+)')
files_paths.sort(key=lambda l: [int(s) if s.isdigit() else s.lower() for s in re.split(dre, l)])
e=0
results = []
for filepath in files_paths:
    imp = IJ.openImage(filepath)
    imp.show()
    d = Duplicator()
    if (imp.getDimensions()[2]<2):
        e += 1
        imp.close()
        continue
    if (imp.getDimensions()[3]>1):
        IJ.run("Z Project...", "projection=[Max Intensity]")
        imp.close()
        imp.show()
        imp= IJ.getImage()

    #Choice of Luts for each chanel
    IJ.resetMinAndMax(imp);
    IJ.run(imp, "Next Slice [>]", "")
    IJ.run(imp, "Green", "")
    IJ.resetMinAndMax(imp);
    IJ.run(imp, "Next Slice [>]", "")
    IJ.run(imp, "Cyan", "")
    IJ.resetMinAndMax(imp);
    imp.setC(1)

    #Reference expmt
    e += 1
    gui=NonBlockingGenericDialog("Experiment?")
    gui.addMessage("Ref+Lamelle")
    gui.addStringField("n°",f'TDP74_2_{e}')
    gui.enableYesNoCancel('ok', 'skip')
    gui.showDialog()

    if gui.wasCanceled():
        imp.close()
        break
    if not gui.wasOKed():
        imp.close()
        continue
    expmt=ij.py.from_java(gui.getNextString())

    #Adjust brighness and contrast
    IJ.run("Brightness/Contrast...")
    gui=NonBlockingGenericDialog("Adjust Brightness/Contrast")
    gui.enableYesNoCancel('ok', 'skip')
    gui.showDialog()

    #Select transfected cells
    splitted_frame=ChannelSplitter.split(imp)
    chGFP_frame=splitted_frame[0]
    chGFP_frame.show()
    IJ.setTool("multipoint")
    WaitForUserDialog("Select Transfected Cells").show()
    rm=RoiManager.getRoiManager()
    transfected_roi=rm.getRoisAsArray()
    rm.runCommand(imp,"Show All");
    if not transfected_roi:
        WaitForUserDialog("Empty ROI Manager. Press 't' key after selecting cells. One more chance:").show()
        rm=RoiManager.getRoiManager()
        transfected_roi=rm.getRoisAsArray()
        transfected=transfected_roi[0].getContainedPoints()
        rm.reset()
    else:
        transfected=transfected_roi[0].getContainedPoints()
        rm.reset()

    IJ.run(chGFP_frame, "Select None", "");

    #Select condensate-displaying cells
    IJ.setTool("multipoint")
    WaitForUserDialog("Select cells with condensates").show()
    condensates_roi=rm.getRoisAsArray()
    rm.runCommand(imp,"Show All");
    if not condensates_roi:
        condensates=[]
        rm.reset()
    else:
        condensates=condensates_roi[0].getContainedPoints()
        rm.reset()
    ratio = len(condensates)/len(transfected)
    results += [[expmt, len(transfected), len(condensates), ratio]]
    display(results)
    imp.close()
    chGFP_frame.close()

total_transfected = 0
total_condensates = 0
for list in results:
    total_condensates += list[2]
    total_transfected += list[1]

results += [['total', total_transfected, total_condensates, total_condensates/total_condensates]]
results_table = pd.DataFrame(results, columns=['expmt', 'Nb of transfected cells', 'Nb of cells with condensates', 'Ratio'])
display(results_table)
results_table.to_excel(r"D:\PhD Pauline\Data\Dissociation\Data_dissociation.xlsx", index=False)

ZeroDivisionError: division by zero

In [14]:
ordered_results = []
i=0
for data in results:
    ordered_results += [results[i:i+4]]
    i += 4
display(ordered_results)

[[['TDP58_1_1', 7, 4, 0.5714285714285714],
  ['TDP58_1_2', 5, 4, 0.8],
  ['TDP58_1_3', 6, 3, 0.5],
  ['TDP58_1_4', 3, 2, 0.6666666666666666]],
 [['TDP58_1_5', 8, 2, 0.25],
  ['TDP58_1_8', 3, 3, 1.0],
  ['TDP58_1_9', 6, 0, 0.0],
  ['TDP58_1_10', 4, 4, 1.0]],
 [['TDP58_1_11', 4, 1, 0.25],
  ['TDP58_1_12', 5, 4, 0.8],
  ['TDP58_1_13', 3, 2, 0.6666666666666666],
  ['TDP58_1_14', 3, 3, 1.0]],
 [['TDP58_1_15', 2, 2, 1.0],
  ['TDP58_1_16', 4, 4, 1.0],
  ['TDP58_1_17', 6, 4, 0.6666666666666666],
  ['TDP58_1_18', 4, 3, 0.75]],
 [['TDP58_1_20', 2, 2, 1.0],
  ['TDP58_1_21', 3, 3, 1.0],
  ['TDP58_1_22', 4, 4, 1.0],
  ['TDP58_1_23', 1, 1, 1.0]],
 [['TDP58_1_24', 6, 6, 1.0],
  ['TDP58_1_25', 4, 2, 0.5],
  ['TDP58_1_26', 2, 1, 0.5],
  ['TDP58_1_27', 5, 1, 0.2]],
 [['TDP58_1_28', 2, 2, 1.0],
  ['TDP58_1_29', 5, 2, 0.4],
  ['TDP58_1_30', 1, 1, 1.0],
  ['TDP58_1_31', 5, 4, 0.8]],
 [['TDP58_1_32', 4, 3, 0.75],
  ['TDP58_1_33', 2, 2, 1.0],
  ['TDP58_1_34', 9, 6, 0.6666666666666666],
  ['TDP58_1_35', 4, 4,

In [7]:
results_table = pd.DataFrame(results, columns=['expmt', 'Nb of transfected cells', 'Nb of cells with condensates', 'Ratio'])
results_table
results_table.to_excel('D:\PhD Pauline\Data\SeedingEffect\Raw_Data.xlsx', index=False)

In [13]:
results

[['TDP58_1_1', 7, 4, 0.5714285714285714],
 ['TDP58_1_2', 5, 4, 0.8],
 ['TDP58_1_3', 6, 3, 0.5],
 ['TDP58_1_4', 3, 2, 0.6666666666666666],
 ['TDP58_1_5', 8, 2, 0.25],
 ['TDP58_1_8', 3, 3, 1.0],
 ['TDP58_1_9', 6, 0, 0.0],
 ['TDP58_1_10', 4, 4, 1.0],
 ['TDP58_1_11', 4, 1, 0.25],
 ['TDP58_1_12', 5, 4, 0.8],
 ['TDP58_1_13', 3, 2, 0.6666666666666666],
 ['TDP58_1_14', 3, 3, 1.0],
 ['TDP58_1_15', 2, 2, 1.0],
 ['TDP58_1_16', 4, 4, 1.0],
 ['TDP58_1_17', 6, 4, 0.6666666666666666],
 ['TDP58_1_18', 4, 3, 0.75],
 ['TDP58_1_20', 2, 2, 1.0],
 ['TDP58_1_21', 3, 3, 1.0],
 ['TDP58_1_22', 4, 4, 1.0],
 ['TDP58_1_23', 1, 1, 1.0],
 ['TDP58_1_24', 6, 6, 1.0],
 ['TDP58_1_25', 4, 2, 0.5],
 ['TDP58_1_26', 2, 1, 0.5],
 ['TDP58_1_27', 5, 1, 0.2],
 ['TDP58_1_28', 2, 2, 1.0],
 ['TDP58_1_29', 5, 2, 0.4],
 ['TDP58_1_30', 1, 1, 1.0],
 ['TDP58_1_31', 5, 4, 0.8],
 ['TDP58_1_32', 4, 3, 0.75],
 ['TDP58_1_33', 2, 2, 1.0],
 ['TDP58_1_34', 9, 6, 0.6666666666666666],
 ['TDP58_1_35', 4, 4, 1.0],
 ['total', 132, 89, 1.0]]